Carga del archivo .csv

In [ ]:
from google.colab import files

# Subir un archivo CSV a Google Colab
uploaded = files.upload()

Saving ventas_electronica.csv to ventas_electronica.csv


In [ ]:
### Pieza de codigo para detectar codificacion del archivo
import chardet

with open('/content/ventas_electronica.csv', 'rb') as file:
  rawdata = file.read()
  result = chardet.detect(rawdata)
  encoding = result['encoding']
  print(f'the detected encoding is (encoding)')

the detected encoding is (encoding)


In [ ]:
### Pieza de codigo para transformar la codificacion 'Windows-1252' del archivo a 'UTF-8'
import codecs

# Abrir el archivo .csv con codificación Windows-1252 ('cp1252')
with codecs.open('/content/ventas_electronica.csv', 'r', encoding='cp1252') as file:
  content = file.read()

# Abrir y sobreescribir con un nuevo archivo .csv con codificación UTF-8
with codecs.open('/content/ventas_electronica.csv','w', encoding='utf-8') as file:
  file.write(content)

Importar bibliotecas

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import scipy.stats

Carga del Dataframe desde archivo CSV

In [ ]:
df = pd.read_csv('/content/ventas_electronica.csv')

Análisis exploratorio:

In [ ]:
print("Tamaño del conjunto de datos: ")
print(df.shape)

print("\nTipos de Datos: ")
print(df.dtypes)

print("Información general del conjunto de datos:")
print(df.info())

print("\n¿Cuántas variables de cada tipo tenemos? ")
print(df.dtypes.value_counts())

In [ ]:
print("Primeras filas del conjunto de datos: ")
print(df.head(5))

In [ ]:
print("Descripción de todas las columnas: ")
df.describe(include='all')

In [ ]:
print("Descripción de las variables numéricas: ")
df.describe(include=[np.number])

In [ ]:
print("Descripción de las variables categóricas: ")
df.describe(include=object)

Análisis de valores nulos:

In [ ]:
print("Valores nulos explícitos: ")
print(df.isnull().any())

print("\nDe tener observaciones con valores nulos, ¿cuántas por cada varible? ")
print(df.isnull().sum())

print("\nValores nulos totales en el conjunto de datos: ")
print(df.isnull().sum().sum())

print("\nProporción de valores nulos por cada variable: ")
df.isnull().melt().pipe(
    lambda f: (
        sns.displot(
            data=f,
            y='variable',
            hue='value',
            multiple='fill',
            aspect=2
        )
    )
)

In [ ]:
print("\nVisualizar valores nulos en todo el conjunto de datos")
df.isnull().transpose().pipe(
    lambda f: (
        sns.heatmap(
            data=f
        )
    )
)

Valores únicos:

In [ ]:
# Valores únicos en cada columna
print("\nValores únicos en cada columna:")
for column in df.columns:
    unique_values = df[column].nunique()
    print(f"{column}: {unique_values} valores únicos")

Visualizaciones exploratorias:

In [ ]:
# Histogramas de las variables
data.hist(bins=20, figsize=(12, 8))
plt.show()

In [ ]:
# Visualizar conteos
custom_colors = ['#0072B5', '#F28424', '#4E2A84', '#34A853', '#FF5900', '#A8A8A8', '#00A1DE', '#E62963', '#43B02A', '#956B9D', '#FF0000']
ax = df['Manufacturer'].value_counts().plot(kind='bar', color=custom_colors)

# Inclinar las etiquetas del eje x en 45 grados
ax.set_xticklabels(ax.get_xticklabels(), rotation=50)

In [ ]:
# Visualización de conteo con Seaborn
print("Visualización de conteo con Seaborn")
ax = df.value_counts('Manufacturer', sort=True).reset_index(name='count').pipe(
    lambda df: (
        sns.barplot(
            data=df,
            x='Manufacturer',
            y='count',
            palette=custom_colors,
            )
        )
    )
ax.set_xticklabels(ax.get_xticklabels(), rotation=50)

Limpieza de datos

In [ ]:
# Eliminamos las columnas innecesarias
df = df.drop("SalesOrderID", axis='columns')
df = df.drop("ProdID", axis='columns')
df = df.drop("StyleName", axis='columns')
df = df.drop("CustomerAccountNumber", axis='columns')
df = df.drop("StoreKey", axis='columns')
df = df.drop("ProductLabel", axis='columns')
df = df.drop("ProductDescription", axis='columns')
df = df.drop("Sale Size", axis='columns')
df = df.drop("Sales Size Sort", axis='columns')
print("Columnas conservadas: ")
print(df.columns)

Columnas conservadas: 
Index(['Country', 'OrderDate', 'SalesChannelCode', 'Manufacturer',
       'Brand Name', 'Class', 'Color', 'StockType', 'Units', 'Sales Amount',
       'NSAT', 'SubCategory', 'Category', 'ProductName', 'Unit Cost',
       'Unit Price', 'SalesTax', 'Freight', 'PurchAgain'],
      dtype='object')


In [ ]:
# Limpiar el símbolo "$" y convertir 'Sales Amount' a entero (int)
df['Sales Amount'] = pd.to_numeric(df['Sales Amount'].str.replace('$', ''), errors='coerce', downcast='integer')

<ipython-input-4-401eb61a92e4>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Sales Amount'] = pd.to_numeric(df['Sales Amount'].str.replace('$', ''), errors='coerce', downcast='integer')


In [ ]:
# Añadir y representar variable categórica de 'sales_channel_code (canal de comercialización) como variable numérica discreta de resultado binario
df = df.assign(
    sales_channel_code=lambda df: df['SalesChannelCode'].replace(['I', 'S'], ['Internet', 'Tienda Física'])
)
value_counts = df['sales_channel_code'].value_counts()
print(value_counts)

In [ ]:
# Convertir la columna de 'Año' a valores numéricos (suponiendo que representa años)
df['OrderDate'] = pd.to_datetime(df['OrderDate'], errors='coerce').dt.year

Visualizaciones estadísticas y de distribución:

In [ ]:
print("Promedios de cada columna del dataframe: ")
print(df.mean().round(2))

print("\nMedia o promedio de Monto de Ventas:" )
print(df['Sales Amount'].mean().round(2))

print("\nMediana de Monto de Ventas: ")
print(df['Sales Amount'].median())

print("\nModa de Monto de Ventas: ")
print(df['Sales Amount'].mode())

In [ ]:
print("Valor máximo de las variables: ")
print(df.max(numeric_only=True))

print("\nValor mínimo de las variables: ")
print(df.min(numeric_only=True))

In [ ]:
print("Intervalos del rango intercuartílico: ")
df.quantile(q=[0.75,0.5,0.25]).transpose().rename_axis('variable').reset_index().assign(
    iqr=lambda df: df[0.75] - df[0.25]
)

In [ ]:
print("Visualizar distribución de una variable: ")
sns.histplot(
    data=df,
    x='Sales Amount'
)

# Línea promedio
plt.axvline(
    x=df['Sales Amount'].mean(),
    color='red',
    linestyle='dashed',
    linewidth=2)

In [ ]:
# Función de probabilidad de masas (PMFs) utilizando seaborn
print("Función de probabilidad de masas: ")
sns.histplot(
    data=df,
    x='Sales Amount',
    binwidth=1,
    stat='probability'
)

In [ ]:
# Importamos la librería empiricaldist para graficar distribuciones
!pip install empiricaldist
import empiricaldist

In [ ]:
# PMFs utilizando empiricaldist
pmf_sales_amount = empiricaldist.Pmf.from_seq(
    df['Sales Amount'],
    normalize=True
)
pmf_sales_amount.bar()

In [ ]:
# Utilizando PMFs para un valor específico
print("Función de probabilidad de masas en el valor '200': ")
print(pmf_sales_amount(200))

# Comprobando valor máximo
print("\nValor máximo de la función de probabilidad de masas: ")
print(pmf_sales_amount.max())

In [ ]:
# Función empírica de probabilidad acumulada (ECDFs)
sns.ecdfplot(
    data=df,
    x='Sales Amount'
)

In [ ]:
# ECDFs utilizando empiricaldist
cdf_sales_amount = empiricaldist.Cdf.from_seq(
    df['Sales Amount'],
    normalize=True
)
cdf_sales_amount.plot()

# Calcular probabilidad acumulada para un valor
q = 200
p = cdf_sales_amount.forward(q)

print("q:", q, "\np:", p, "\n")

# Añadiendo líneas guía
plt.vlines(
    x=q,
    ymin=0,
    ymax=p,
    color='black',
    linestyle='dashed'
)

plt.hlines(
    y=p,
    xmin=pmf_sales_amount.qs[0],
    xmax=q,
    color='black',
   linestyle='dashed'
)

plt.plot(q, p, 'ro')

In [ ]:
# ECDFs con quantiles
cdf_sales_amount.step()
p_1 = 0.25 # Specify probability
p_2 = 0.75
ps = (0.25, 0.75) # IQR
qs = cdf_sales_amount.inverse(ps)
plt.vlines(
    x=qs,
    ymin=0,
    ymax=ps,
    color='black',
    linestyle='dashed'
)

plt.hlines(
   y=ps,
   xmin=pmf_sales_amount.qs[0],
   xmax=qs,
   color='black',
   linestyle='dashed'
)

plt.scatter(
   x=qs,
   y=ps,
   color='red',
   zorder=2
)

In [ ]:
# Comparar distribuciones
sns.ecdfplot(
    data=df,
    x='Sales Amount',
    hue='Country',
    palette=custom_colors
)

In [ ]:
# Función de densidad de probabilidad -PMF-
sns.kdeplot(
    data=df,
    x='Sales Amount',
    bw_method=0.1
)

In [ ]:
# Estadísticas de la variable 'Sales Amount'
stats = df['Sales Amount'].describe()
stats

In [ ]:
# Función de distribución acumulada de una variable -CDF-
xs = np.linspace(stats['min'], stats['max'])
ys = scipy.stats.norm(stats['mean'], stats['std']).cdf(xs)

# Visualizando la función CDF:
plt.plot(xs, ys, color='black', linestyle='dashed')

empiricaldist.Cdf.from_seq(
    df['Sales Amount'],
    normalize=True
).plot()

In [ ]:
# Función de densidad de probabilidad de una variable -PDF-
xs = np.linspace(stats['min'], stats['max'])
ys = scipy.stats.norm(stats['mean'], stats['std']).pdf(xs)

# Visualizando la función PDF:
plt.plot(xs, ys, color='black', linestyle='dashed')

sns.kdeplot(
    data=df,
    x='Sales Amount'
)

Estableciendo relaciones:

In [ ]:
# Gráfica de puntos
sns.scatterplot(
    data=df,
    x='Country',
    y='Sales Amount',
    hue='sales_channel_code',
    alpha=1/2, #transparencia
    s=5 #tamaño de los puntos
)

In [ ]:
# Histograma 2D
g = sns.displot(
    data=df,
    x='Country',
    y='Sales Amount',
    hue='sales_channel_code',
    rug=True
    )
g.set_axis_labels('Country', 'Sales Amount')
plt.xticks(rotation=50)

# Verificar la cantidad de datos antes de la división


In [ ]:
print("Número total de filas:", len(df))



# Verificar los tipos de datos después de la conversión


In [ ]:
print(df.dtypes)

# Preparar los datos


In [ ]:
X = df['OrderDate'].values.reshape(-1, 1)  # Característica: Orderdate
y = df['SalesOrderID'].values  # Variable objetivo: SalesOrderID

# Dividir los datos en conjuntos de entrenamiento y prueba


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear un modelo de regresión lineal


In [ ]:
model = LinearRegression()

# Entrenar el modelo con los datos de entrenamiento


In [ ]:
model.fit(X_train, y_train)

# Realizar predicciones


In [ ]:
predictions = model.predict(X_test)

# Visualizar los resultados


In [ ]:
plt.scatter(X_test, y_test, color='blue', label='Datos reales')
plt.plot(X_test, predictions, color='red', label='Regresión lineal')
plt.title('Regresión Lineal de Ventas en función del OrderDate')
plt.xlabel('OrderDate')
plt.ylabel('SalenOrderID')
plt.legend()
plt.show()